In [113]:
import tensorflow as tf
import pandas as pd
import sklearn

In [114]:
#Initializing 
tf.reset_default_graph()


In [115]:
acts_rep_table = pd.read_csv('../data/acts_rep_table.csv')

In [116]:
acts_rep_table.head()

label     1     2   3  4  5  6     7  8     9 ...  23  24  25  26  27  28  \
0      2  8648  8648  10  2  2  3  8148  0  3692 ...   0   0   0   0   0   0   
1      7  8648     2  10  2  2  3  8148  0  3692 ...   0   0   0   0   0   0   
2      2  8648  8648  10  2  2  3  8148  0  3692 ...   0   0   0   0   0   0   
3      7  8648  8648  10  2  2  3  8148  0  3692 ...   0   0   0   0   0   0   
4      7  8648  8648  10  2  2  3  8148  0  3692 ...   0   0   0   0   0   0   

   29  30  31  32  
0   0   0   0   0  
1   0   0   0   0  
2   0   0   0   0  
3   0   0   0   0  
4   0   0   0   0  

[5 rows x 33 columns]

In [117]:
INPUT_DIM = X.shape[1] #Input dimensions
H_SIZE = 10 #Number of hidden layer neurons
ETA = 1e-2 #Learning Rate
OUTPUT_DIM = y.max() + 1

In [118]:
input_x = tf.placeholder(tf.float32, [None,INPUT_DIM] , name="input_x")

W1 = tf.get_variable("W1", shape=[INPUT_DIM, H_SIZE],
           initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(input_x,W1))
W2 = tf.get_variable("W2", shape=[H_SIZE, OUTPUT_DIM],
           initializer=tf.contrib.layers.xavier_initializer())
y_predicted = tf.matmul(layer1,W2)


In [119]:
input_y = tf.placeholder(tf.float32,[None,OUTPUT_DIM], name="input_y")


In [120]:
ce1 = tf.nn.softmax_cross_entropy_with_logits(logits=y_predicted, labels=input_y)
adam = tf.train.AdamOptimizer(learning_rate=ETA)
train_step = adam.minimize(ce1)

In [121]:
correct_prediction = tf.equal(tf.argmax(y_predicted, 1), tf.argmax(input_y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [132]:
BATCH_SIZE = 10
BATCHES = 5000

def feed_dict_from_df(bdf):
    xs, ys = [], []
    X = bdf.drop('label', axis = 1)
    y = bdf.label
    for i in range(bdf.shape[0]):
        xs.append(X.iloc[i].values)
        y_hot = [0]*OUTPUT_DIM
        y_hot[y.iloc[i]] = 1
        ys.append(y_hot)
    return xs, ys
        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for batch in range(BATCHES):
        bdf = acts_rep_table.sample(BATCH_SIZE)
        xs, ys = feed_dict_from_df(bdf)
        sess.run(train_step, feed_dict={input_x: xs, input_y: ys})
        if batch % (BATCHES/10) == 0:
            train_accuracy = accuracy.eval(feed_dict={input_x: xs, input_y: ys})
            print('batch %d, training accuracy %g' % (batch, train_accuracy))
    xs, ys = feed_dict_from_df(acts_rep_table)
    test_accuracy = accuracy.eval(feed_dict={input_x: xs, input_y: ys})
    print('overall accuracy %g' % (test_accuracy))


batch 0, training accuracy 0.2
batch 500, training accuracy 0.1
batch 1000, training accuracy 0.1
batch 1500, training accuracy 0
batch 2000, training accuracy 0
batch 2500, training accuracy 0.1
batch 3000, training accuracy 0.1
batch 3500, training accuracy 0
batch 4000, training accuracy 0
batch 4500, training accuracy 0
overall accuracy 0.0488722
